[View in Colaboratory](https://colab.research.google.com/github/sleepless-se/python_reference/blob/master/RNN_alice_pub_001.ipynb)

## 概要

このプログラムはアリス・イン・ワンダーランド（英語版）をディープラーニングで学習させ、その学習データを元に文章の生成を行います。

### 使い方

1. このファイルは閲覧専用です。「ファイル → ドライブにコピーを保存」で実行できるようになります。

2. 書籍：アリス・イン・ワンダーランド（UTF-8）をここからダウンロードします。

 http://www.gutenberg.org/files/11/11-0.txt

 保存時のファイル名は「alice.txt」にしてください。


3. [3] でサーバーにアップロードします。

※ プログラムは各行「Shift＋Enter」で実行できます。

（上から連打していけばあとは勝手に実行されます。）

In [1]:
from keras.layers import Dense , Activation , SimpleRNN 
from keras.models import Sequential 
import numpy as np


Using TensorFlow backend.


In [0]:
from google.colab import files

In [3]:
file = files.upload()
!ls

Saving alice.txt to alice.txt
alice.txt  datalab  sample_data


In [4]:
FILE_PATH = "./alice.txt"
text=""
with open(FILE_PATH, 'r',encoding="utf-8") as f:
  for line in f:
    lines = line.split()
    text += " ".join(lines)
text = text.lower()
text

'\ufeffproject gutenberg’s alice’s adventures in wonderland, by lewis carrollthis ebook is for the use of anyone anywhere at no cost and withalmost no restrictions whatsoever. you may copy it, give it away orre-use it under the terms of the project gutenberg license includedwith this ebook or online at www.gutenberg.orgtitle: alice’s adventures in wonderlandauthor: lewis carrollposting date: june 25, 2008 [ebook #11]release date: march, 1994last updated: october 6, 2016language: englishcharacter set encoding: utf-8*** start of this project gutenberg ebook alice’s adventures in wonderland ***alice’s adventures in wonderlandlewis carrollthe millennium fulcrum edition 3.0chapter i. down the rabbit-holealice was beginning to get very tired of sitting by her sister on thebank, and of having nothing to do: once or twice she had peeped into thebook her sister was reading, but it had no pictures or conversations init, ‘and what is the use of a book,’ thought alice ‘without pictures orconversat

In [0]:
chars = set(text)
nb_chars = len(chars)
char2index = dict((c,i) for i , c in enumerate(chars))
index2char = dict((i,c) for i , c in enumerate(chars))

In [0]:
SEQLEN = 10
STEP = 1

input_chars = []
label_chars = []

for i in range( 0,len(text) - SEQLEN , STEP ):
  input_chars.append(text[i:i+SEQLEN])  
  label_chars.append(text[i+SEQLEN])  
  
  

In [0]:
X = np.zeros((len(input_chars),SEQLEN,nb_chars),dtype=np.bool)
Y = np.zeros((len(input_chars),nb_chars),dtype=np.bool)
for i , input_char in enumerate(input_chars):
  for j , ch in enumerate(input_char):
    X[i,j,char2index[ch]] = 1
  Y[i,char2index[label_chars[i]]] = 1
    


次の行を実行すると学習と学習過程で生成される文章を確認できます。

文章が少しずつきれいになっていく様子をお楽しみください。

※ Generating form sead の次の行が生成された文章です。

In [9]:
HIDDEN_SIZE = 128 
BATCH_SIZE = 128 
NUM_ITERATIONS = 50 
NUM_EPOCHS_PER_ITERATION = 1 
NUM_PREDS_PER_EPOCH = 100 

model = Sequential() 
model.add(SimpleRNN( HIDDEN_SIZE , return_sequences = False , input_shape = ( SEQLEN , nb_chars ),unroll = True )) 
model.add( Dense ( nb_chars )) 
model.add( Activation ( "softmax" )) 
model.compile( loss = "categorical_crossentropy" , optimizer = "rmsprop" )

for iteration in range(NUM_ITERATIONS):
  print("=" * 50)
  print("Iteration #: %d" % (iteration))
  model.fit(X,Y,batch_size=BATCH_SIZE,epochs=NUM_EPOCHS_PER_ITERATION)
  test_index = np.random.randint(len(input_chars))
  test_chars = input_chars[test_index]
  print("Generating form sead: {}".format(test_chars))
  print(test_chars , end="")
  for i in range(NUM_PREDS_PER_EPOCH):
    Xtest = np.zeros((1,SEQLEN,nb_chars))
    for i , ch in enumerate(test_chars):
      Xtest[0,i,char2index[ch]]=1
    pred = model.predict(Xtest,verbose=0)[0]
    ypred = index2char[np.argmax(pred)]
    print(ypred,end="")
    test_chars = test_chars[1:] + ypred
  print() 
   

Iteration #: 0
Epoch 1/1
158772/158772 [==============================] - 10s 60us/step - loss: 2.4339
Generating form sead:  plates, a
 plates, and the therereand the southe tore therereand the southe tore therereand the southe tore therereand t
Iteration #: 1
Epoch 1/1
158772/158772 [==============================] - 9s 59us/step - loss: 2.1409
Generating form sead: work very 
work very un the said the said the said the said the said the said the said the said the said the said the sai
Iteration #: 2
Epoch 1/1
158772/158772 [==============================] - 9s 59us/step - loss: 2.0474
Generating form sead: oves nothi
oves nothing the wast on the wast on the wast on the wast on the wast on the wast on the wast on the wast on t
Iteration #: 3
Epoch 1/1
158772/158772 [==============================] - 9s 59us/step - loss: 1.9728
Generating form sead: er folding
er folding and the mage project gutenberg-tm the she was she was she was she was she was she was she was she w
Iteration #: 4



こちらのサイトからタイトルを選択し、

http://www.gutenberg.org/browse/scores/top

「Plain Text UTF-8」をクリックすると今回使用した「アリス・イン・ワンダーランド」と同様にファイルをダウンロードでき、［３］でアップロードをすると選択したタイトル風の文章の生成ができます。

いろいろためして、遊んでみてくださいね(^^)
